# Identification of NI using method 3 (EM admission)

Notes: - Includes data from 1st January 2020 - 31st March 2021

In [3]:
# Importing Libraries
import import_ipynb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Importing Notebooks
# import NI_M2

In [4]:

SaveCSV = False
ReadExcel = True

# Optional: Reading Excel file and saving csv file
if SaveCSV:
    read_file = pd.read_excel(r'O:\DiRAC\FlavienHardy\2_NosocomialInfections\Data\0_Raw\MarchData\All episodes 07-06-21.xlsx', sheet_name='Sheet1')
    read_file.to_csv(r'O:\DiRAC\FlavienHardy\2_NosocomialInfections\Data\1_Processed\All episodes 07-06-21.csv', index=None, header=True)

if ReadExcel:
    # Importing dataset
    df_Readmission = pd.read_excel(r'O:\DiRAC\FlavienHardy\2_NosocomialInfections\Data\0_Raw\MarchData\Readmission within 10 days 18-06-21.xlsx')


In [6]:
df_Readmission.head()

,P_Spell_ID,Epikey,diagnosis_group,Admission_date,Discharge_date,HESID,P_Spell_ID1,Admission_date1,Discharge_date1,FinY,POD,Spell_Los,diagnosis_group1,TimeBetween,COVID FIRST
0,146800259,700773439562,T812~U071~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~...,2021-01-07,2021-02-26,AA9BB2835BF0B6B076C0F96D3DE16E77,146800258,2020-10-12,2021-01-07,2020/21,EM,87,K914~T810~Y833~A411~J90X~Y833~J981~Y833~B968~U...,0,0
1,39135714,700781722214,U071~J128~B972~Z518~J90X~I951~I480~E274~E86X~E...,2020-03-23,2020-04-16,2D5C0AC4A78C8CD041982BBF15ABED80,39135713,2020-03-06,2020-03-20,2019/20,EM,14,J189~Y95X~N179~E876~R464~E86X~S000~W192~N189~J...,3,0
2,180748140,510256641500,U071~R060~R05X~C911~F059~D509~D510~J449~C349~C...,2020-03-23,2020-03-25,D21F2487D4DBBFD35F162B62BDCA443B,180748139,2020-02-20,2020-03-17,2019/20,EM,26,M2556~W100~J449~I10X~E119~C61X~Z851~-1~-1~-1~-...,6,0
3,180748140,510256641432,U071~R060~R05X~F059~D509~D510~J449~C349~C911~C...,2020-03-23,2020-03-25,D21F2487D4DBBFD35F162B62BDCA443B,180748139,2020-02-20,2020-03-17,2019/20,EM,26,M2556~W100~J449~I10X~E119~C61X~Z851~-1~-1~-1~-...,6,0
4,180748140,510256641464,U071~R060~R05X~F059~C911~D509~D510~J449~C349~C...,2020-03-23,2020-03-25,D21F2487D4DBBFD35F162B62BDCA443B,180748139,2020-02-20,2020-03-17,2019/20,EM,26,M2556~W100~J449~I10X~E119~C61X~Z851~-1~-1~-1~-...,6,0


## Pre-processing

In [62]:

df_M3 = df_Readmission

# Keeping unique spells
df_FirstAdmission = df_M3.drop_duplicates(subset=['P_Spell_ID'], keep='first')
df_FirstAdmission = df_FirstAdmission.sort_values(by=['P_Spell_ID'])
df_FirstAdmission.head(2)


,P_Spell_ID,Epikey,diagnosis_group,Admission_date,Discharge_date,HESID,P_Spell_ID1,Admission_date1,Discharge_date1,FinY,POD,Spell_Los,diagnosis_group1,TimeBetween,COVID FIRST
109607,27772,700778119076,J128~B972~N390~K590~E119~G35X~I454~E662~E669~F...,2020-11-20,2020-12-02,0008399E344FC517C7DE6458055F2BA3,27771,2020-10-21,2020-11-13,2020/21,EM,23,J189~J90X~J9691~E119~G35X~R53X~E669~E876~F329~...,7,0
50670,39241,700776960596,A415~U071~K830~K804~K802~N179~N189~J128~B972~Y...,2020-03-08,2020-04-24,000B85A69F9649A61FF5BD86B92B17A0,39240,2020-02-26,2020-03-04,2019/20,EM,7,A419~K804~J181~Y95X~J90X~N184~I447~E113D~H360A...,4,0


In [63]:

df  = df_FirstAdmission

# Keep infections with lOS>N days
Ndays = 15
FilterLOS = df['TimeBetween']>=Ndays
df['NC_Method2_15'] = np.where(FilterLOS, 1, 0)
df[df['NC_Method2_15']==1].head()


,P_Spell_ID,Epikey,diagnosis_group,Admission_date,Discharge_date,HESID,P_Spell_ID1,Admission_date1,Discharge_date1,FinY,POD,Spell_Los,diagnosis_group1,TimeBetween,COVID FIRST,NC_Method2_15


In [5]:

# Selecting columns
ColumnList = ['P_Spell_ID', 'HESID', 'diagnosis_group', 'Admission_date', 'Discharge_date', 'total time 15']
df_M3 = df_Readmission[ColumnList]
df_M3.loc['NC_Method3']=df_M3['total time 15']
df_M3 = df_M3.drop(columns={'total time 15'})
df_M3.head(2)


KeyError: "['total time 15'] not in index"